# Vector Index Performance Comparison: HNSW vs IVF

이 노트북은 PostgreSQL pgvector의 HNSW와 IVF 인덱스 성능을 비교합니다.

In [1]:
import asyncio
import time
import numpy as np
from sqlalchemy import text, select
from sqlalchemy.ext.asyncio import AsyncSession
import sys
import os

from dotenv import load_dotenv, find_dotenv

# Load environment variables
dotenv_path = find_dotenv()
print(f"Target .env path: {dotenv_path}")
loaded = load_dotenv(dotenv_path, override=True)
print(f"Loading successful: {loaded}")

# Add project root to path
sys.path.append(os.path.abspath('../..'))

from app.db.database import AsyncSessionLocal, engine
from app.models.document import DocumentChunk
from app.core.config import settings

Target .env path: /Users/kimjunghyeon/Desktop/workspace/ai-agent/.env
Loading successful: True


## 1. 인덱스 상태 확인

In [2]:
async def check_indexes():
    """현재 존재하는 벡터 인덱스들을 확인합니다."""
    async with AsyncSessionLocal() as session:
        # pgvector 인덱스 확인
        query = text("""
            SELECT 
                schemaname,
                tablename, 
                indexname,
                indexdef
            FROM pg_indexes 
            WHERE tablename = 'document_chunks' 
                AND indexname LIKE '%embedding%'
            ORDER BY indexname;
        """)
        
        result = await session.execute(query)
        indexes = result.fetchall()
        
        print("=== Vector Indexes ===")
        for idx in indexes:
            print(f"Name: {idx.indexname}")
            print(f"Definition: {idx.indexdef}")
            print("---")
        
        # 테이블 통계
        stats_query = text("""
            SELECT 
                COUNT(*) as total_chunks,
                COUNT(embedding) as chunks_with_embedding,
                COUNT(DISTINCT document_id) as unique_documents
            FROM document_chunks;
        """)
        
        stats = await session.execute(stats_query)
        row = stats.fetchone()
        
        print("=== Table Statistics ===")
        print(f"Total chunks: {row.total_chunks:,}")
        print(f"Chunks with embedding: {row.chunks_with_embedding:,}")
        print(f"Unique documents: {row.unique_documents:,}")
        
        return indexes

# 인덱스 상태 확인
indexes = await check_indexes()

=== Vector Indexes ===
Name: ix_document_chunks_embedding_hnsw
Definition: CREATE INDEX ix_document_chunks_embedding_hnsw ON public.document_chunks USING hnsw (embedding vector_cosine_ops)
---
Name: ix_document_chunks_embedding_ivf
Definition: CREATE INDEX ix_document_chunks_embedding_ivf ON public.document_chunks USING ivfflat (embedding vector_cosine_ops)
---
Name: ix_document_chunks_with_embedding
Definition: CREATE INDEX ix_document_chunks_with_embedding ON public.document_chunks USING btree (document_id, embedding_model) WHERE (embedding IS NOT NULL)
---
=== Table Statistics ===
Total chunks: 362
Chunks with embedding: 362
Unique documents: 2


## 2. 성능 테스트 함수 정의

In [3]:
async def get_random_embedding():
    """랜덤 쿼리 벡터를 위해 기존 임베딩 중 하나를 선택합니다."""
    async with AsyncSessionLocal() as session:
        query = text("""
            SELECT embedding 
            FROM document_chunks 
            WHERE embedding IS NOT NULL 
            ORDER BY RANDOM() 
            LIMIT 1;
        """)
        
        result = await session.execute(query)
        row = result.fetchone()
        return row.embedding if row else None

async def test_vector_search_performance(index_type: str, query_embedding, k: int = 10, iterations: int = 10):
    """특정 인덱스를 사용한 벡터 검색 성능을 테스트합니다."""
    
    times = []
    
    async with AsyncSessionLocal() as session:
        # PostgreSQL 설정 (seqscan 비활성화)
        await session.execute(text("SET enable_seqscan = off"))
        await session.execute(text("SET enable_indexscan = on"))
        
        # 벡터 검색 쿼리
        query = text("""
            SELECT id, content, embedding <=> :query_embedding as distance
            FROM document_chunks 
            WHERE embedding IS NOT NULL
            ORDER BY embedding <=> :query_embedding
            LIMIT :k;
        """)
        
        for i in range(iterations):
            start_time = time.time()
            
            result = await session.execute(query, {
                'query_embedding': query_embedding,
                'k': k
            })
            
            rows = result.fetchall()
            end_time = time.time()
            
            times.append(end_time - start_time)
            
            if i == 0:  # 첫 번째 실행에서 결과 개수 확인
                result_count = len(rows)
    
    return {
        'index_type': index_type,
        'k': k,
        'iterations': iterations,
        'times': times,
        'avg_time': np.mean(times),
        'min_time': np.min(times),
        'max_time': np.max(times),
        'std_time': np.std(times),
        'result_count': result_count
    }

async def get_query_execution_plan(index_type: str, query_embedding, k: int = 10):
    """쿼리 실행 계획을 확인합니다."""
    async with AsyncSessionLocal() as session:
        # EXPLAIN ANALYZE로 실제 실행 계획 확인
        explain_query = text("""
            EXPLAIN (ANALYZE, BUFFERS, FORMAT JSON)
            SELECT id, content, embedding <=> :query_embedding as distance
            FROM document_chunks 
            WHERE embedding IS NOT NULL
            ORDER BY embedding <=> :query_embedding
            LIMIT :k;
        """)
        
        result = await session.execute(explain_query, {
            'query_embedding': query_embedding,
            'k': k
        })
        
        plan = result.fetchone()[0]
        return plan

async def force_index_usage(index_name: str, query_embedding, k: int = 10):
    """특정 인덱스를 강제로 사용하도록 하는 테스트 함수"""
    async with AsyncSessionLocal() as session:
        if 'hnsw' in index_name.lower():
            # HNSW 인덱스에 최적화된 설정
            await session.execute(text("SET enable_seqscan = off"))
            await session.execute(text("SET enable_indexscan = on"))
            await session.execute(text("SET enable_bitmapscan = off"))
        else:
            # IVF 인덱스에 최적화된 설정
            await session.execute(text("SET enable_seqscan = off")) 
            await session.execute(text("SET enable_indexscan = on"))
            await session.execute(text("SET enable_bitmapscan = on"))
        
        query = text("""
            SELECT id, content, embedding <=> :query_embedding as distance
            FROM document_chunks 
            WHERE embedding IS NOT NULL
            ORDER BY embedding <=> :query_embedding
            LIMIT :k;
        """)
        
        start_time = time.time()
        result = await session.execute(query, {
            'query_embedding': query_embedding,
            'k': k
        })
        rows = result.fetchall()
        end_time = time.time()
        
        return {
            'execution_time': end_time - start_time,
            'result_count': len(rows),
            'index_hint': index_name
        }

## 3. 성능 비교 실행

In [4]:
# 테스트용 쿼리 벡터 준비
query_embedding = await get_random_embedding()

if query_embedding is None:
    print("⚠️  임베딩이 있는 청크가 없습니다. 먼저 문서를 업로드하고 임베딩을 생성해주세요.")
else:
    print(f"✅ 쿼리 벡터 준비 완료 (차원: {len(query_embedding)})")
    
    # 다양한 k 값에 대해 성능 테스트
    k_values = [5, 10, 20, 50, 100]
    results = []
    
    print("\n=== 성능 테스트 시작 ===")
    
    for k in k_values:
        print(f"\nTesting k={k}...")
        
        # HNSW 테스트
        print("  - HNSW index...")
        hnsw_result = await test_vector_search_performance('hnsw', query_embedding, k=k, iterations=20)
        results.append(hnsw_result)
        
        # IVF 테스트  
        print("  - IVF index...")
        ivf_result = await test_vector_search_performance('ivf', query_embedding, k=k, iterations=20)
        results.append(ivf_result)
        
        print(f"    HNSW: {hnsw_result['avg_time']*1000:.2f}ms (±{hnsw_result['std_time']*1000:.2f})")
        print(f"    IVF:  {ivf_result['avg_time']*1000:.2f}ms (±{ivf_result['std_time']*1000:.2f})")
    
    print("\n✅ 성능 테스트 완료!")

✅ 쿼리 벡터 준비 완료 (차원: 12790)

=== 성능 테스트 시작 ===

Testing k=5...
  - HNSW index...
  - IVF index...
    HNSW: 1.96ms (±1.12)
    IVF:  1.40ms (±0.58)

Testing k=10...
  - HNSW index...
  - IVF index...
    HNSW: 1.14ms (±0.41)
    IVF:  0.90ms (±0.23)

Testing k=20...
  - HNSW index...
  - IVF index...
    HNSW: 1.42ms (±0.94)
    IVF:  0.76ms (±0.15)

Testing k=50...
  - HNSW index...
  - IVF index...
    HNSW: 1.99ms (±1.34)
    IVF:  1.89ms (±0.31)

Testing k=100...
  - HNSW index...
  - IVF index...
    HNSW: 2.40ms (±0.58)
    IVF:  2.02ms (±0.22)

✅ 성능 테스트 완료!


## 4. 결과 분석 및 시각화

In [5]:
# 결과를 딕셔너리 리스트로 변환하고 직접 출력
result_data = [
    {
        'Index Type': r['index_type'].upper(),
        'K': r['k'],
        'Avg Time (ms)': round(r['avg_time'] * 1000, 2),
        'Min Time (ms)': round(r['min_time'] * 1000, 2),
        'Max Time (ms)': round(r['max_time'] * 1000, 2),
        'Std Time (ms)': round(r['std_time'] * 1000, 2),
        'Results': r['result_count']
    } for r in results
]

print("=== Performance Comparison Results ===")
print(f"{'Index Type':<12} {'K':<5} {'Avg(ms)':<10} {'Min(ms)':<10} {'Max(ms)':<10} {'Std(ms)':<10} {'Results':<8}")
print("-" * 70)
for data in result_data:
    print(f"{data['Index Type']:<12} {data['K']:<5} {data['Avg Time (ms)']:<10} {data['Min Time (ms)']:<10} {data['Max Time (ms)']:<10} {data['Std Time (ms)']:<10} {data['Results']:<8}")

# 성능 비교 분석
print("\n=== Performance Analysis ===")
hnsw_results = [d for d in result_data if d['Index Type'] == 'HNSW']
ivf_results = [d for d in result_data if d['Index Type'] == 'IVF']

for i, k in enumerate([5, 10, 20, 50, 100]):
    hnsw_time = hnsw_results[i]['Avg Time (ms)']
    ivf_time = ivf_results[i]['Avg Time (ms)']
    improvement = (ivf_time - hnsw_time) / ivf_time * 100
    print(f"K={k}: HNSW {hnsw_time}ms vs IVF {ivf_time}ms → {improvement:.1f}% improvement")

=== Performance Comparison Results ===
Index Type   K     Avg(ms)    Min(ms)    Max(ms)    Std(ms)    Results 
----------------------------------------------------------------------
HNSW         5     1.96       0.75       5.52       1.12       5       
IVF          5     1.4        0.69       2.13       0.58       5       
HNSW         10    1.14       0.7        2.0        0.41       5       
IVF          10    0.9        0.61       1.54       0.23       5       
HNSW         20    1.42       0.6        4.27       0.94       5       
IVF          20    0.76       0.58       1.09       0.15       5       
HNSW         50    1.99       1.51       7.78       1.34       50      
IVF          50    1.89       1.56       2.78       0.31       50      
HNSW         100   2.4        1.88       4.49       0.58       100     
IVF          100   2.02       1.81       2.69       0.22       100     

=== Performance Analysis ===
K=5: HNSW 1.96ms vs IVF 1.4ms → -40.0% improvement
K=10: HNSW 1.14ms

In [6]:
# 간단한 텍스트 기반 시각화
print("=== Visual Performance Comparison ===")

print("\n📊 Average Response Time by K value:")
print(f"{'K Value':<8} {'HNSW (ms)':<12} {'IVF (ms)':<12} {'Improvement':<12}")
print("-" * 50)

hnsw_results = [d for d in result_data if d['Index Type'] == 'HNSW']
ivf_results = [d for d in result_data if d['Index Type'] == 'IVF']

k_values = [5, 10, 20, 50, 100]
for i, k in enumerate(k_values):
    hnsw_time = hnsw_results[i]['Avg Time (ms)']
    ivf_time = ivf_results[i]['Avg Time (ms)']
    improvement = (ivf_time - hnsw_time) / ivf_time * 100
    print(f"{k:<8} {hnsw_time:<12} {ivf_time:<12} {improvement:+.1f}%")

# ASCII 바 차트
print("\n📈 Performance Improvement (HNSW vs IVF):")
for i, k in enumerate(k_values):
    hnsw_time = hnsw_results[i]['Avg Time (ms)']
    ivf_time = ivf_results[i]['Avg Time (ms)']
    improvement = (ivf_time - hnsw_time) / ivf_time * 100
    
    # 바 길이 계산 (최대 50칸)
    bar_length = max(0, int(improvement * 50 / 100))
    bar = "█" * bar_length + "░" * (50 - bar_length)
    
    print(f"K={k:>3}: {bar} {improvement:+.1f}%")

print("\n📋 Summary Statistics:")
hnsw_avg = sum(r['Avg Time (ms)'] for r in hnsw_results) / len(hnsw_results)
ivf_avg = sum(r['Avg Time (ms)'] for r in ivf_results) / len(ivf_results)
overall_improvement = (ivf_avg - hnsw_avg) / ivf_avg * 100

print(f"  • Overall HNSW Average: {hnsw_avg:.2f} ms")
print(f"  • Overall IVF Average: {ivf_avg:.2f} ms")
print(f"  • Overall Improvement: {overall_improvement:+.1f}%")

=== Visual Performance Comparison ===

📊 Average Response Time by K value:
K Value  HNSW (ms)    IVF (ms)     Improvement 
--------------------------------------------------
5        1.96         1.4          -40.0%
10       1.14         0.9          -26.7%
20       1.42         0.76         -86.8%
50       1.99         1.89         -5.3%
100      2.4          2.02         -18.8%

📈 Performance Improvement (HNSW vs IVF):
K=  5: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -40.0%
K= 10: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -26.7%
K= 20: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -86.8%
K= 50: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -5.3%
K=100: ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -18.8%

📋 Summary Statistics:
  • Overall HNSW Average: 1.78 ms
  • Overall IVF Average: 1.39 ms
  • Overall Improvement: -27.8%


## 5. 쿼리 실행 계획 분석

In [7]:
# 쿼리 실행 계획 확인 (k=10으로 테스트)
print("=== Query Execution Plans ===")

print("\n--- HNSW Index Plan ---")
hnsw_plan = await get_query_execution_plan('hnsw', query_embedding, k=10)
print(f"Execution Time: {hnsw_plan[0]['Execution Time']:.3f} ms")
print(f"Planning Time: {hnsw_plan[0]['Planning Time']:.3f} ms")

print("\n--- IVF Index Plan ---")
ivf_plan = await get_query_execution_plan('ivf', query_embedding, k=10)
print(f"Execution Time: {ivf_plan[0]['Execution Time']:.3f} ms")
print(f"Planning Time: {ivf_plan[0]['Planning Time']:.3f} ms")

# 상세 실행 계획 출력 (선택사항)
import json
print("\n=== Detailed HNSW Plan ===")
print(json.dumps(hnsw_plan[0]['Plan'], indent=2))

print("\n=== Detailed IVF Plan ===")
print(json.dumps(ivf_plan[0]['Plan'], indent=2))

=== Query Execution Plans ===

--- HNSW Index Plan ---
Execution Time: 0.179 ms
Planning Time: 0.063 ms

--- IVF Index Plan ---
Execution Time: 0.088 ms
Planning Time: 0.046 ms

=== Detailed HNSW Plan ===
{
  "Node Type": "Limit",
  "Parallel Aware": false,
  "Async Capable": false,
  "Startup Cost": 11.6,
  "Total Cost": 51.08,
  "Plan Rows": 10,
  "Plan Width": 1026,
  "Actual Startup Time": 0.148,
  "Actual Total Time": 0.161,
  "Actual Rows": 5.0,
  "Actual Loops": 1,
  "Disabled": false,
  "Shared Hit Blocks": 127,
  "Shared Read Blocks": 0,
  "Shared Dirtied Blocks": 0,
  "Shared Written Blocks": 0,
  "Local Hit Blocks": 0,
  "Local Read Blocks": 0,
  "Local Dirtied Blocks": 0,
  "Local Written Blocks": 0,
  "Temp Read Blocks": 0,
  "Temp Written Blocks": 0,
  "Plans": [
    {
      "Node Type": "Index Scan",
      "Parent Relationship": "Outer",
      "Parallel Aware": false,
      "Async Capable": false,
      "Scan Direction": "Forward",
      "Index Name": "ix_document_chunks

## 6. 권장사항 및 결론

In [8]:
# 결론 및 권장사항 생성
hnsw_avg_time = sum(r['Avg Time (ms)'] for r in result_data if r['Index Type'] == 'HNSW') / 5
ivf_avg_time = sum(r['Avg Time (ms)'] for r in result_data if r['Index Type'] == 'IVF') / 5
improvement = (ivf_avg_time - hnsw_avg_time) / ivf_avg_time * 100

print("=== Performance Analysis Summary ===")
print(f"\n📊 Overall Results:")
print(f"  • HNSW Average: {hnsw_avg_time:.2f} ms")
print(f"  • IVF Average: {ivf_avg_time:.2f} ms")
print(f"  • HNSW Improvement: {improvement:.1f}%")

print(f"\n🎯 Recommendations:")
if improvement > 20:
    print(f"  • ✅ HNSW shows significant performance improvement ({improvement:.1f}%)")
    print(f"  • 🔄 Consider removing IVF index to save storage space")
    print(f"  • 📈 HNSW is recommended for production use")
elif improvement > 5:
    print(f"  • ✅ HNSW shows moderate improvement ({improvement:.1f}%)")
    print(f"  • ⚖️  Keep both indexes for now, monitor real-world performance")
else:
    print(f"  • ⚠️  Performance difference is minimal ({improvement:.1f}%)")
    print(f"  • 🔍 Consider data size, query patterns, and hardware factors")

print(f"\n⚙️  Next Steps:")
sample_results = result_data[0]['Results'] if result_data else 0
print(f"  1. Test with larger datasets (current: {sample_results} results)")
print(f"  2. Test with different distance functions (cosine vs L2)")
print(f"  3. Monitor real-world query performance")
print(f"  4. Consider index maintenance costs")

# 인덱스 크기 비교
async with AsyncSessionLocal() as session:
    size_query = text("""
        SELECT 
            indexname,
            pg_size_pretty(pg_relation_size(indexname::regclass)) as size
        FROM pg_indexes 
        WHERE tablename = 'document_chunks' 
            AND indexname LIKE '%embedding%'
        ORDER BY pg_relation_size(indexname::regclass) DESC;
    """)
    
    result = await session.execute(size_query)
    sizes = result.fetchall()
    
    print(f"\n💾 Index Storage Usage:")
    for size_info in sizes:
        print(f"  • {size_info.indexname}: {size_info.size}")

=== Performance Analysis Summary ===

📊 Overall Results:
  • HNSW Average: 1.78 ms
  • IVF Average: 1.39 ms
  • HNSW Improvement: -27.8%

🎯 Recommendations:
  • ⚠️  Performance difference is minimal (-27.8%)
  • 🔍 Consider data size, query patterns, and hardware factors

⚙️  Next Steps:
  1. Test with larger datasets (current: 5 results)
  2. Test with different distance functions (cosine vs L2)
  3. Monitor real-world query performance
  4. Consider index maintenance costs

💾 Index Storage Usage:
  • ix_document_chunks_embedding_ivf: 3704 kB
  • ix_document_chunks_embedding_hnsw: 2904 kB
  • ix_document_chunks_with_embedding: 16 kB


## 결론

이 성능 비교를 통해 HNSW와 IVF 인덱스의 특성을 확인할 수 있습니다:

### HNSW (Hierarchical Navigable Small World)
- ✅ 일반적으로 더 빠른 검색 성능
- ✅ 높은 정확도 유지
- ❌ 더 많은 메모리 사용
- ❌ 빌드 시간이 더 오래 걸림

### IVF (Inverted File)
- ✅ 메모리 효율적
- ✅ 빠른 인덱스 빌드
- ❌ 상대적으로 느린 검색
- ❌ 정확도가 약간 낮을 수 있음

### 실제 프로덕션 결정 요소
1. **데이터 크기**: 큰 데이터셋에서 HNSW 이점이 더 명확
2. **쿼리 빈도**: 높은 QPS에서 HNSW 권장
3. **메모리 제약**: 제한적인 경우 IVF 고려
4. **정확도 요구사항**: 높은 정확도 필요시 HNSW